DISNEY DATASET CREATION

Developing program idea on 1 example movie

In [45]:
#Task1. Get info box

from bs4 import BeautifulSoup as bs
import requests

In [ ]:
#Load demo webpage
r = requests.get("https://en.wikipedia.org/wiki/Pinocchio_(1940_film)")

#Convert to a beautiful soup object
soup = bs(r.content)

#Print out the HTML
contents = soup.prettify()
print(contents)

In [ ]:
info_box = soup.find(class_="infobox vevent")  # finding object in HTML code that has class name "infobox vevent"
info_box.prettify()

print(info_box)

In [ ]:
info_rows = info_box.find_all("tr")   #choosing all rows containing "tr"

for row in info_rows:
    print(row.prettify())             #in the list getting all HTML lines.

In [49]:
import re

# function to create data dictionary
def get_data_dictionary(info_box):
    #saving into a dictionary
    movie_info = {}

  
    info_rows = info_box.find_all("tr")   #choosing all rows containing "tr"

    for index, row in enumerate(info_rows):
        if index == 0:
            movie_info['Title'] = row.find("th").get_text(" ", strip=True)

        elif index == 1:
            continue  

        elif row.find('span', class_='bday dtstart published updated'):
            content_key = row.find("th").get_text(" ", strip=True)

            # Extract the date values and collect them in a list
            content_data = []
            for span in row.find_all('span', class_='bday dtstart published updated'):
                content_data.append(span.get_text(" ",strip=True))

            for index in range(len(content_data)):
                    item = content_data[index]
                    content_data[index] = re.sub(r'\s*\[.*?\]', '', item)
            
            movie_info[content_key] = content_data
            
            
         #checking for table data that contains multiple elemets
        elif row.find("li"):
            content_key = row.find("th").get_text(" ", strip=True)

            # Extract all table values and collect them in a list
            content_data = []
            for li in row.find_all("li"):
                content_data.append(li.get_text(" ", strip=True))
                        
                for i, element in enumerate(content_data):
                    # replacing all "\xa0"s with space
                    if "\xa0" in element:
                        content_data[i] = element.replace("\xa0", " ")    

            # finding possible refernces from HTML and removing them
            content_data = [re.sub(r'\s*\[.*?\]', '', item) for item in content_data]

            movie_info[content_key] = content_data

        # default
        else:
            content_key = row.find("th").get_text(" ", strip=True)
                
            content_data = row.find("td")
            content_data = content_data.get_text(" ", strip=True)

            # finding possible refernces from HTML
            for i in range(10):
                str_item = f"[{i}]"
                if str_item in content_data:
                    content_data = content_data.replace(str_item, "")

            movie_info[content_key] = content_data

    return movie_info

In [ ]:
movie = get_data_dictionary(info_box)
movie["Release dates"]

WORKING ON WALT_DISNAY WEBPAGE

Applying method to create data dictionary before on all of the webpages

In [ ]:
#Load webpage - Toy story
r = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")

#Convert to a beautiful soup object
soup =bs(r.content)

#Print out the HTML
contents = soup.prettify()
print(contents)

In [ ]:
#getting rows that contains wikipedia URL
container = soup.select(".wikitable.sortable i")
filtered_container = [row for row in container if "/wiki" in str(row)]

filtered_container

In [53]:
# getting URL of webpages
def get_movie_url(row):

    head = 'https://en.wikipedia.org/'
    url = row.a['href']

    url = head + url

    return url

# using URL getting InfoBox of webpage
def get_info_box(url):
    # Load webpage
    r = requests.get(url)

    # Convert to a beautiful soup object
    soup = bs(r.content)

    info_box = soup.find(class_="infobox vevent")

    return info_box

In [ ]:
# list to store whole data from all webpages
movie_data = []

for row in filtered_container:
    #using try-except block to avoid possible errors (if possible solve them)
    try:
        url = get_movie_url(row)
        info_box = get_info_box(url)
        dictionary = get_data_dictionary(info_box)

        movie_data.append(dictionary)
    except Exception as e:
        print(row.get_text())
        print(e)

Save/Reload final data list in json data file form

In [55]:
import json

def save_data_json(title, data):
    with open(title, 'w', encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

def load_data_json(title):
    with open(title, encoding='utf-8') as f:
        data = json.load(f)
        return data

In [56]:
save_data_json("disney_movies_data.json", movie_data)

Data Cleaning

In [ ]:
data = load_data_json("disney_movies_data.json")
len(data)

In [58]:
from datetime import datetime
import re

def convert_date_format(old_date): 
    if isinstance(old_date, datetime):
        return old_date
    
    elif isinstance(old_date, str):
        if old_date is not None:
            # checking for type YYYY-MM-DD
            pattern = r"\d{4}-\d{2}-\d{2}"

            if re.match(pattern, old_date):
                new_date = datetime.strptime(old_date, "%Y-%m-%d").date()
                return new_date

            # checking for type YYYY-MM
            pattern = r"\d{4}-\d{2}"

            if re.match(pattern, old_date):
                new_date = datetime.strptime(old_date, "%Y-%d").date()
                return new_date

            else:
                # Separate and change elements in the date
                separated_date = old_date.split()

                # To delete elements having "(", ")", and "["
                filtered_date = [elem for elem in separated_date if "(" not in elem and ")" not in elem and "[" not in elem]

                # Removing commas from elements
                filtered_date = [elem.replace(',', '') for elem in filtered_date]

                # List of months
                month = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

                # Changing name of months to corresponding number
                for i in range(len(filtered_date)):
                    if filtered_date[i] in month:
                        filtered_date[i] = str(month.index(filtered_date[i]) + 1)

                year_index = []

                # Getting indexes of years in filtered_date list
                for index, elm in enumerate(filtered_date):
                    if elm not in month:
                        if len(elm) == 4:
                            year_index.append(index)

                temp_sublist = []
                sublist = []

                # Separating filtered_date into lists that contain the actual date in the desired format
                for index, elm in enumerate(filtered_date):
                    sublist.append(elm)
                    if index in year_index:
                        temp_sublist.append(sublist)
                        sublist = []

                if sublist:
                    temp_sublist.append(sublist)

                new_date = []

                # Making format of date YYYY-MM-DD
                for elm in temp_sublist:
                    # For YYYY-MM-DD
                    if len(elm) == 3:
                        date = elm[2] + "-" + elm[0].zfill(2) + "-" + elm[1].zfill(2)
                        new_date.append(date)
                    # For YYYY-MM
                    elif len(elm) == 2:
                        date = elm[1] + "-" + elm[0].zfill(2)
                        new_date.append(date)
                    elif len(elm) == 1:
                        new_date.append(elm[0])
                    else:
                        # Handle the case where the list is empty or has more than 3 elements
                        new_date.append("N/A")

                # Final checking for the format of the date. If it contains strange things, remove it
                valid_dates = []
                for date in new_date:
                    if re.match(r"\d{4}-\d{2}-\d{2}", date) or re.match(r"\d{4}-\d{2}", date):
                        valid_dates.append(date)

                if valid_dates:
                    # checking for type YYYY-MM-DD
                    pattern = r"\d{4}-\d{2}-\d{2}"

                    if re.match(pattern, valid_dates[0]):
                        return datetime.strptime(valid_dates[0], "%Y-%m-%d").date()
                    else:
                        return datetime.strptime(valid_dates[0], "%Y-%d").date()
                    
                else:
                    return None
        else:
            return None
    else:
        if old_date is None:
            return None
        else:
            for elm in old_date:
                if elm != "N/A" and elm is not None:
                    new_date = convert_date_format(elm)
                    return new_date
                else:
                    return None

In [59]:
for movie in data:
    old_date = movie.get("Release dates")
    new_date = convert_date_format(old_date)
    movie["Release dates"] = new_date

In [ ]:
print(len(data))
for movie in data:
    date = movie.get("Release dates")
    if date is not None:
        print(type(date))

In [61]:
import pickle

def save_data_pickle(data, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(data, file)

def load_data_pickle(file_path):
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    return data

In [62]:
# date type can not be saved in json file, so use pickle file
save_data_pickle(data, "disney_movies_updated.pickle")

In [ ]:
# converting Running time values form "80 minutues" structure to "80" integer value
data_updated = load_data_pickle("disney_movies_updated.pickle")

def convert_RunningTime_format(run_time):
    if run_time is not None:
        run_time = run_time.split()
        return int(run_time[0])
    else:
        return "N/A"

for movie in data_updated:
    run_time = movie.get("Running time")
    time = convert_RunningTime_format(run_time)
    movie["Running time"] = time

    movie["Running time (min)"] = movie.pop("Running time")

print(len(data_updated))

In [64]:
# date type can not be saved in json file, so use pickle file
save_data_pickle(data_updated, "disney_movies_updated.pickle")

In [65]:
# converting format of budget
def convert_currency_format(budget):
    if budget is None:
        return "N/A"
    
    else:
        try:
            if isinstance(budget, str):

                # removing text in ( )
                budget = re.sub(r'\([^)]*\)', '', budget)
                # removing text in [ ]
                budget = re.sub(r'\[[^\]]*\]', '', budget)

                budget_seperated = budget.split()

                if len(budget_seperated) == 1:
                    amount = budget_seperated[0].replace("$", "").replace(">", "").replace("<", "")
                    budget_updated = float(amount.replace(",", ""))
                    return budget_updated
                
                if len(budget_seperated) == 2:
                    
                    if "million" in budget_seperated:
                        amount = budget_seperated[0]
                        
                        # checking for currency:
                        if "AU$" in amount:
                            budget_updated = round(float(amount.replace("AU$", "").replace(">", "")) * 750000, 2)
                            return budget_updated
                        
                        elif "A$" in amount:
                            budget_updated = round(float(amount.replace("A$", "").replace(">", "")) * 750000, 2)
                            return budget_updated
                        
                        elif "-" in amount:
                            # removing "$" sign and seperating string by "-"
                            amount = amount.replace("$", "").replace(">", "")
                            amount = amount.split("-")

                            budget_updated = (float(amount[0]) + float(amount[1])) * 500000
                            
                            return budget_updated
                        
                        elif "–" in amount:
                            # removing "$" sign and seperating string by "-"
                            amount = amount.replace("–", "-")
                            amount = amount.replace("$", "").replace(">", "")
                            amount = amount.split("-")

                            budget_updated = (float(amount[0]) + float(amount[1])) * 500000
                            
                            return budget_updated

                        else:
                            budget_updated = float(amount.replace("$", "").replace(">", "")) * 1000000
                            return budget_updated
                        
                    elif "crore" in budget_seperated:
                        amount = budget_seperated[0]
                        budget_updated = float(amount.replace("₹", "").replace(">", "")) * 130000

                        return budget_updated
                
                # for cases like there are more than 2 word in input string
                else:
                    # checking for case when there is "–" in the input
                    for elm in budget_seperated:
                        if "–" in elm:
                            amount = []
                            for index, elm in enumerate(budget_seperated):
                                # Regular expression pattern for any digit
                                pattern = r'\d'
                                match = re.search(pattern, elm)

                                # finding elm that contains any numerical values
                                if match is not None:
                                    if index < len(budget_seperated) -1:
                                        # if next element is "million", then multiplying by 1000000
                                        if budget_seperated[index+1] == "million":
                                            
                                            price = float(elm.replace("$", "").replace("–", "").replace(">", "")) * 1000000
                                            

                                            amount.append(price)

                                        else:
                                            price = float(elm.replace("$", "").replace("–", "").replace(",", "").replace(">", ""))
                                            
                                            amount.append(price)

                            budget_updated = 0

                            # finding sum of prices
                            for item in amount:
                                budget_updated += item
                            
                            # finding average of prices
                            budget_updated = budget_updated / len(amount)

                            return budget_updated

                    # checking for cases like there is "or" in the string
                    if "or" in budget_seperated:
                        # list to store numerical values
                        
                        amount = []

                        for index, elm in enumerate(budget_seperated):
                            # Regular expression pattern for any digit
                            pattern = r'\d'
                            match = re.search(pattern, elm)

                            # finding elm that contains any numerical values
                            if match is not None:
                                # if next element is "million", then multiplying by 1000000
                                if index < len(budget_seperated)-1:
                                    if budget_seperated[index+1] == "million":
                                        price = float(elm.replace("$", "").replace(">", "")) * 1000000
                                    
                                        amount.append(price)

                                # else just getting float type amount
                                else:
                                    price = elm.replace("$", "").replace(">", "")
                                    price = float(price.replace(",", ""))
                                    
                                
                                    amount.append(price)

                        budget_updated = 0

                        # to sum all items in amount list
                        for item in amount:
                            budget_updated += item

                        # find average of items
                        budget_updated = budget_updated / 2

                        return budget_updated

                    # checking for cases like there is "to" in the string
                    elif "to" in budget_seperated:
                        # list to store numerical values
                        amount = []

                        for elm in budget_seperated:
                            # Regular expression pattern for any digit
                            pattern = r'\d'
                            match = re.search(pattern, elm)

                            if match is not None:
                                amount.append(elm)
                        
                        # if there is million in the list, multiplying by 1000000
                        if "million" in budget_seperated:
                            for index, item in enumerate(amount):
                                amount[index] = float(item.replace("$", "").replace(">", "")) * 1000000


                            budget_updated = 0

                            # getting sum of amount items
                            for item in amount:
                                budget_updated += item
                            
                            # getting average of amounts in the list
                            budget_updated = budget_updated / len(amount)

                            return budget_updated
                        
                        else:
                            # removing signs
                            for index, item in enumerate(amount):
                                amount[index] = item.replace("$", "").replace(">", "")
                                amount[index] = float(amount.replace(",", ""))

                            budget_updated = 0

                            # getting sum of amount items
                            for item in amount:
                                budget_updated += item

                            # getting average of amounts in the list
                            budget_updated = budget_updated / len(amount)

                            return budget_updated
                        
                    else:
                        return "N/A"

            elif isinstance(budget, list):
                list_amount = []

                for elm in budget:
                    # removing text in ( )
                    elm = re.sub(r'\([^)]*\)', '', elm)
                    # removing text in [ ]
                    elm = re.sub(r'\[[^\]]*\]', '', elm)
                    
                    elm_seperated = elm.split()

                    # taking amount number
                    amount = elm_seperated[0]

                    amount = float(amount.replace("$", "").replace(">", "")) * 1000000

                    list_amount.append(amount)

                budget_updated = 0

                for item in list_amount:
                    budget_updated += item

                budget_updated = budget_updated / len(list_amount)

                return budget_updated

        except Exception as e:
            print(budget)
            print(e)

In [ ]:
data_updated = load_data_pickle("disney_movies_updated.pickle")

for movie in data_updated :
    budget = movie.get("Budget")
    new_budget = convert_currency_format(budget)

    movie["Budget"] = new_budget

    movie["Budget ($)"] = movie.pop("Budget")

print(len(data_updated ))

In [67]:
save_data_pickle(data_updated, "disney_movies_updated.pickle")

In [ ]:
data_updated = load_data_pickle("disney_movies_updated.pickle")

for movie in data_updated:
    box_office = movie.get("Box office")
    new_box_office = convert_currency_format(box_office)
    movie["Box office"] = new_box_office

In [69]:
for movie in data_updated:
    box_office = movie.get("Box office")
    movie["Box office ($)"] = movie.pop("Box office")

In [70]:
save_data_pickle(data, "disney_movies_final_data.pickle")

In [ ]:
pickle_data = load_data_pickle("disney_movies_final_data.pickle")

pickle_data[10]

In [ ]:
import pandas as pd

df = pd.DataFrame(pickle_data)

df.head()

In [78]:
df.to_csv("disney_movies_final_data.csv")